# Решение СЛАУ с помощью метода Гаусса и LU-разложения

##  Метод Гаусса — Вариант 1: прямой метод

In [3]:
def gauss_elimination_v1(matrix):
    """
    Решает систему линейных уравнений методом Гаусса
    Модифицирует исходную матрицу на месте
    Ожидается, что матрица содержит дополнительную строку из нулей для корректной работы
    """
    # Прямой ход
    for i in range(0, len(matrix) - 1):
        pivot = matrix[i][i]
        for col in range(len(matrix[0])):
            matrix[i][col] /= pivot
        for row in range(i + 1, len(matrix) - 1):
            factor = matrix[row][i]
            for col in range(len(matrix[0])):
                matrix[row][col] -= matrix[i][col] * factor

    # Удаляем строку из нулей
    matrix.remove([0, 0, 0, 0])

    # Обратный ход (нахождение переменных)
    x3 = matrix[2][3] / matrix[2][2]
    x2 = (matrix[1][3] - matrix[1][2] * x3) / matrix[1][1]
    x1 = (matrix[0][3] - matrix[0][2] * x3 - matrix[0][1] * x2) / matrix[0][0]
    print(f'x1 = {x1}, x2 = {x2}, x3 = {x3}')


## Метод Гаусса — Вариант 2: с копированием в новую матрицу

In [4]:
def gauss_elimination_v2(original_matrix):
    """
    Второй способ решения СЛАУ методом Гаусса
    Копирует данные в новую матрицу и проводит пошаговое обнуление ниже главной диагонали
    """
    # Создаем пустую матрицу
    matrix = [[0, 0, 0, 0] for _ in range(3)]

    # Нормализуем первую строку
    pivot = original_matrix[0][0]
    for col in range(len(original_matrix[0])):
        matrix[0][col] = original_matrix[0][col] / pivot

    # Обнуляем элементы под главной диагональю
    for step in range(1, len(original_matrix)):
        for row in range(step, len(original_matrix)):
            for col in range(step, len(original_matrix[0])):
                if col == 0:
                    matrix[row][col] = 0
                elif step == 2:
                    matrix[row][col] = matrix[row][col] - (matrix[row][step - 1] * matrix[step - 1][col]) / matrix[step - 1][step - 1]
                else:
                    matrix[row][col] = original_matrix[row][col] - (original_matrix[row][step - 1] * original_matrix[step - 1][col]) / original_matrix[step - 1][step - 1]

            # Нормализуем текущую строку
            pivot = matrix[row][step]
            if step != 2 and row < len(original_matrix) - 1:
                for j in range(len(matrix[row])):
                    matrix[row][j] /= pivot

    # Обратный ход (нахождение переменных)
    x3 = matrix[2][3] / matrix[2][2]
    x2 = (matrix[1][3] - matrix[1][2] * x3) / matrix[1][1]
    x1 = (matrix[0][3] - matrix[0][2] * x3 - matrix[0][1] * x2) / matrix[0][0]
    print(f'x1 = {x1}, x2 = {x2}, x3 = {x3}')


## Пример использования:

In [5]:
# Матрица для первого метода (с дополнительной строкой нулей)
matrix1 = [
    [5, 0, 1, 11],
    [2, 6, -2, 8],
    [-3, 2, 10, 6],
    [0, 0, 0, 0]
]

# Матрица для второго метода
matrix2 = [
    [2, 1, 4, 16],
    [3, 2, 1, 10],
    [1, 3, 3, 16]
]

gauss_elimination_v1(matrix1)
gauss_elimination_v2(matrix2)


x1 = 2.0, x2 = 1.0, x3 = 1.0000000000000002
x1 = 1.0, x2 = 2.0, x3 = 3.0


#  LU-разложение и решение СЛАУ

In [6]:
import numpy as np

def lu_decompose(matrix):
    """
    Выполняет LU-разложение матрицы
    Возвращает две матрицы: нижнюю (L) и верхнюю (U)
    """
    n = len(matrix)
    L = np.zeros((n, n))
    U = np.zeros((n, n))

    for i in range(n):
        # Верхняя треугольная матрица
        for k in range(i, n):
            sum_upper = sum(L[i][j] * U[j][k] for j in range(i))
            U[i][k] = matrix[i][k] - sum_upper

        # Нижняя треугольная матрица
        for k in range(i, n):
            if i == k:
                L[i][i] = 1
            else:
                sum_lower = sum(L[k][j] * U[j][i] for j in range(i))
                L[k][i] = (matrix[k][i] - sum_lower) / U[i][i]

    return L, U


## Решение уравнения с помощью LU-разложения

In [7]:
def solve_using_lu(matrix_A, vector_b):
    """
    Решает СЛАУ, используя LU-разложение
    matrix_A — коэффициенты,
    vector_b — свободные члены
    """
    L, U = lu_decompose(matrix_A)
    n = len(matrix_A)

    # Прямой ход (Ly = b)
    y = np.zeros(n)
    for i in range(n):
        y[i] = vector_b[i] - sum(L[i][j] * y[j] for j in range(i))

    # Обратный ход (Ux = y)
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - sum(U[i][j] * x[j] for j in range(i + 1, n))) / U[i][i]

    return x


## Пример использования LU-метода

In [8]:
A = np.array([
    [2, 1, 4],
    [3, 2, 1],
    [1, 3, 3]
])

b = np.array([16, 10, 16])

solution = solve_using_lu(A, b)

# Вывод результата
result_string = ""
for idx, value in enumerate(solution):
    result_string += f"x{idx} = {value}, "

print(result_string)


x0 = 1.0, x1 = 2.0, x2 = 3.0, 
